In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/My Drive/train (1).csv')

In [ ]:
print(df)

       emotion                                             pixels
0            0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1            0  151 150 147 155 148 133 111 140 170 174 182 15...
2            2  231 212 156 164 174 138 161 173 182 200 106 38...
3            4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4            6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...
...        ...                                                ...
34882        6  50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...
34883        3  178 174 172 173 181 188 191 194 196 199 200 20...
34884        0  17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...
34885        3  30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...
34886        2  19 13 14 12 13 16 21 33 50 57 71 84 97 108 122...

[34887 rows x 2 columns]


In [ ]:
num_features = 64
num_labels = 7
batch_size = 64
epochs = 100
width, height = 48, 48

datapoints = df['pixels'].tolist()

#getting features for training
X = []
for xseq in datapoints:
    xx = [int(xp) for xp in xseq.split(' ')]
    xx = np.asarray(xx).reshape(width, height)
    X.append(xx.astype('float32'))

X = np.asarray(X)
X = np.expand_dims(X, -1)

#getting labels for training
y = pd.get_dummies(df['emotion']).to_numpy()

#storing them using numpy
np.save('fdataX', X)
np.save('flabels', y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=76)


In [ ]:
print(X_train)

[[[[ 26.]
   [ 15.]
   [ 15.]
   ...
   [ 59.]
   [ 93.]
   [124.]]

  [[ 17.]
   [ 15.]
   [  9.]
   ...
   [ 14.]
   [ 29.]
   [107.]]

  [[ 16.]
   [ 19.]
   [  4.]
   ...
   [ 32.]
   [ 81.]
   [ 77.]]

  ...

  [[184.]
   [186.]
   [188.]
   ...
   [ 57.]
   [ 41.]
   [ 84.]]

  [[185.]
   [187.]
   [190.]
   ...
   [ 34.]
   [ 60.]
   [ 92.]]

  [[189.]
   [192.]
   [194.]
   ...
   [ 37.]
   [ 81.]
   [ 93.]]]


 [[[246.]
   [246.]
   [242.]
   ...
   [164.]
   [155.]
   [157.]]

  [[241.]
   [240.]
   [230.]
   ...
   [160.]
   [193.]
   [200.]]

  [[240.]
   [236.]
   [221.]
   ...
   [229.]
   [227.]
   [226.]]

  ...

  [[252.]
   [253.]
   [253.]
   ...
   [216.]
   [232.]
   [234.]]

  [[253.]
   [253.]
   [253.]
   ...
   [254.]
   [252.]
   [253.]]

  [[254.]
   [254.]
   [254.]
   ...
   [253.]
   [254.]
   [253.]]]


 [[[255.]
   [218.]
   [111.]
   ...
   [ 67.]
   [ 70.]
   [ 93.]]

  [[255.]
   [192.]
   [104.]
   ...
   [ 65.]
   [ 75.]
   [ 85.]]

  [[254.]
   [15

In [ ]:
model = Sequential()

model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(width, height, 1), data_format='channels_last', kernel_regularizer=l2(0.01)))
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(2*2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*num_features, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 46, 46, 64)        36928     
_________________________________________________________________
batch_normalization (BatchNo (None, 46, 46, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 23, 23, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 23, 23, 128)       73856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 23, 23, 128)       5

In [ ]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])



In [ ]:
#training the model
model.fit(np.array(X_train), np.array(y_train),
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(np.array(X_test), np.array(y_test)),
          shuffle=True)

Epoch 1/100
437/437 [==============================] - 28s 45ms/step - loss: 2.3066 - accuracy: 0.1951 - precision_1: 0.1970 - recall_1: 0.0248 - val_loss: 1.8058 - val_accuracy: 0.2489 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 2/100
437/437 [==============================] - 19s 44ms/step - loss: 1.8418 - accuracy: 0.2460 - precision_1: 0.2473 - recall_1: 0.0014 - val_loss: 1.8227 - val_accuracy: 0.2528 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 3/100
437/437 [==============================] - 19s 44ms/step - loss: 1.8029 - accuracy: 0.2666 - precision_1: 0.4010 - recall_1: 0.0054 - val_loss: 1.8257 - val_accuracy: 0.2488 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 4/100
437/437 [==============================] - 19s 44ms/step - loss: 1.7088 - accuracy: 0.3059 - precision_1: 0.6434 - recall_1: 0.0602 - val_loss: 1.6027 - val_accuracy: 0.3750 - val_precision_1: 0.8896 - val_recall_1: 0.0797
Epoch 5/100
437/437 [===============

In [ ]:
model.save('model.h5')
print("Model saved as h5")

Model saved as h5
